In [1]:
"""
# Chapter 10 Data Aggregation and Group Operations.
Categorizing a dataset and applying a function to each group, whether an aggreation or transformation,
is a critical componet of a data analysis workflow. 

# GroupBy Mechanics: - Split - Apply - Combine -
In the first stage of the process:
data contained in a pandas object is spit into groups based on one or more keys that you provide.
The splittting is performed on a particular axis of an object,
then after a function is applied to each group, producing a new value.
Resulting object kind if dependent of mutation to data.
"""

%matplotlib notebook
# Financial Time Series 
import numpy as np 
import pandas as pd
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

In [2]:
df  = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                    'key2' : ['One', 'two', 'one', 'two', 'one'],
                    'data1' : np.random.randn(5),
                    'data2' : np.random.randn(5)
                })
display(df)

# TO compute the mean of the data1 column suing the labels from key1. 
grouped = df['data1'].groupby(df['key1'])

display(grouped.std())

means = df['data1'].groupby([df['key1'], df['key2']]).mean()

# Here we grouped the data using two keys, and the resulting series now has a hierarchial index consisting of the unique of keys observed
  
display(means)

display(means.unstack())

# Regardless of the objective in using groupby, a generally useful GroupBy method is size, which reurns a Series containing group sizes:

display(df.groupby(['key1', 'key2']).size())

,key1,key2,data1,data2
0,a,One,-0.144112,-0.512520
1,a,two,1.920710,-0.478941
2,b,one,0.805816,2.507690
3,b,two,0.013982,-0.996579
4,a,one,-0.118414,2.367331


key1
a    1.184777
b    0.559911
Name: data1, dtype: float64

key1  key2
a     One    -0.144112
      one    -0.118414
      two     1.920710
b     one     0.805816
      two     0.013982
Name: data1, dtype: float64

key2,One,one,two
key1,,,
a,-0.144112,-0.118414,1.920710
b,NaN,0.805816,0.013982


key1  key2
a     One     1
      one     1
      two     1
b     one     1
      two     1
dtype: int64

In [3]:
"""
# Iterating Over Groups: The GroupBy object supports iteration,
genarating a sequence of 2-tuples containing  the group  name along with the chuck of data. 
"""

for name, group in df.groupby('key1'):
    print(name)
    print(group)
    
        
# In the case of multiple keys, the first element in the tuple of key values:

for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)
    
#  Of course,you can choose to do whaetever with thee pieces of data.
# Computing a dict of the data pieces as a one liner:

pieces = dict(list(df.groupby('key1')))

display(pieces['b'])

# By default groupby groups on axis =0, but you can hroup on any of the other axes.

a
  key1 key2     data1     data2
0    a  One -0.144112 -0.512520
1    a  two  1.920710 -0.478941
4    a  one -0.118414  2.367331
b
  key1 key2     data1     data2
2    b  one  0.805816  2.507690
3    b  two  0.013982 -0.996579
('a', 'One')
  key1 key2     data1    data2
0    a  One -0.144112 -0.51252
('a', 'one')
  key1 key2     data1     data2
4    a  one -0.118414  2.367331
('a', 'two')
  key1 key2    data1     data2
1    a  two  1.92071 -0.478941
('b', 'one')
  key1 key2     data1    data2
2    b  one  0.805816  2.50769
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.013982 -0.996579


,key1,key2,data1,data2
2,b,one,0.805816,2.507690
3,b,two,0.013982,-0.996579


In [4]:
"""
# Selecting a Column or Subset of Columns:
It may be desirable to aggregate only a fre columns. 

To compute means for just the data2 column and get the reslut as a df 
"""

s_grouped = df.groupby(['key1', 'key2'])['data2']

display(s_grouped.mean())


key1  key2
a     One    -0.512520
      one     2.367331
      two    -0.478941
b     one     2.507690
      two    -0.996579
Name: data2, dtype: float64

In [10]:
"""
# Grouping with Dicts andf Series

Grouping information may exist in a form other than an array. 

"""
people = pd.DataFrame(np.random.randn(5, 5),
                     columns=['a', 'b', 'c', 'd', 'e'],
                     index=['Joe', 'Steven', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1,2]] = np.nan

display(people)

# Suppose a group corrspondence for the columns and want to sum togehter the clumns by group

mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

by_column = people.groupby(mapping, axis=1)

display(by_column.sum())

# The same fucntionality holds for Serues, which can be viewd as a fixed size-mapping

map_serries = pd.Series(mapping)

display(map_serries)

display(people.groupby(map_serries, axis=1).count())

,a,b,c,d,e
Joe,-0.519065,0.741475,-0.574417,-1.433153,0.338756
Steven,-0.448253,-0.924646,1.342309,-0.053818,-1.463022
Wes,-0.001318,NaN,NaN,-0.602676,-0.357464
Jim,1.930604,-1.307524,-0.445912,-0.651534,-0.302983
Travis,-0.635849,1.631111,-0.427774,1.534683,1.532533


,blue,red
Joe,-2.007570,0.561166
Steven,1.288491,-2.835921
Wes,-0.602676,-0.358781
Jim,-1.097446,0.320097
Travis,1.106909,2.527794


a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

,blue,red
Joe,2,3
Steven,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [14]:
"""
# Grouping with Functions

Using Python finctions is a more generic way
of defiening a group mapping campared with a dict or Series.
Any functions passed as a group key will be called once per index value, with the retun being
used as the group names.


To group by the length of the names; while you could compute an array of string lengths,
it's simpler to just pass the len function
"""

display(people.groupby(len).sum())

# Mixing functions with arrays, dicts, or Series is not a problem
# as everything gets converted to arrays intenally 

key_list = ['one', 'one', 'one', 'two','two']

display(people.groupby([len, key_list]).min())

,a,b,c,d,e
3,1.410221,-0.566049,-1.020329,-2.687363,-0.321690
6,-1.084102,0.706464,0.914535,1.480865,0.069511


a         b         c         d         e
3 one -0.519065  0.741475 -0.574417 -1.433153 -0.357464
  two  1.930604 -1.307524 -0.445912 -0.651534 -0.302983
6 one -0.448253 -0.924646  1.342309 -0.053818 -1.463022
  two -0.635849  1.631111 -0.427774  1.534683  1.532533

In [22]:
"""
# Data Agregations: Any data transformation that produces scalar values from arrays.
Many common aggregations, such as those found in page 302 have optimized implementations.
"""

display(df)

grouped = df.groupby('key1')

display(grouped['data1'].quantile(0.9))

# To use your own aggregation fucntions, pass any fucntion that
# agregates an arrat to the aggregate or agg method:

def peak_to_peak(arr):
    return arr.max() - arr.min()

display(grouped.agg(peak_to_peak))

display(grouped.describe())

,key1,key2,data1,data2
0,a,One,-0.144112,-0.512520
1,a,two,1.920710,-0.478941
2,b,one,0.805816,2.507690
3,b,two,0.013982,-0.996579
4,a,one,-0.118414,2.367331


key1
a    1.512885
b    0.726633
Name: data1, dtype: float64

C:\Users\JBarr\anaconda3\envs\dev\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,data1,data2
key1,,
a,2.064822,2.879851
b,0.791834,3.504269


data1                                                                       data2                                                                      
     count      mean       std       min       25%       50%       75%       max count      mean       std       min       25%       50%       75%       max
key1                                                                                                                                                        
a      3.0  0.552728  1.184777 -0.144112 -0.131263 -0.118414  0.901148  1.920710   3.0  0.458623  1.653075 -0.512520 -0.495731 -0.478941  0.944195  2.367331
b      2.0  0.409899  0.559911  0.013982  0.211941  0.409899  0.607858  0.805816   2.0  0.755556  2.477892 -0.996579 -0.120512  0.755556  1.631623  2.507690

In [29]:
"""
# Column-Wise Multiple Funtion Application
# adding a tip pct columns
"""

tips = pd.read_csv('tips.csv')

tips['tip_pct'] = tips['tip'] / tips['total_bill']

display(tips)

# To aggregate using a different fucntion depending on hte column, or multiple fucntions at once.

grouped = tips.groupby(['day', 'smoker'])


,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240
6,8.77,2.00,Male,No,Sun,Dinner,2,0.228050
7,26.88,3.12,Male,No,Sun,Dinner,4,0.116071
8,15.04,1.96,Male,No,Sun,Dinner,2,0.130319
9,14.78,3.23,Male,No,Sun,Dinner,2,0.218539


In [46]:
# For descriptive statistics like those in table 10-1 use fucntion name as string

grouped_pct = grouped['tip_pct']

display(grouped_pct.agg('mean'))

# passing a list of fucntions or function names instead,
# u get bacj a DataFrame with column namess taken from the functions:

display(grouped_pct.agg(['mean', 'std', peak_to_peak]))

# Passing a list of (name, fucntion) tuples, 
# the first element of each tuple will be used as the DataFrame column names 
# (Think of a list of 2 tuples as an ordered mapping)

display(grouped_pct.agg([('foo', 'mean'), ('bar', np.std)]))

# With D DataFrame You have more options,specify a list of functions 
# to apply to all of the columns or different functions per column.
# Suppose we wanted to compute the same three statistics for the tip_pct and total bill

fucntions = ['count', 'mean', 'max']

result = grouped['tip_pct', 'total_bill'].agg(fucntions)

display(result)

display(result['tip_pct'])

# As before, a list of tuples with custom names can be passed

ftuples = [('Durchschnitt', 'mean'), ('Abeweichung', np.var)]

display(grouped['tip_pct', 'total_bill'].agg(ftuples))

# To apply potentially different functions to one or more of the columns.
# To do this, pass a dict to agg that contains a mapping of column names to any of the function 
# specification listed so far:

display(grouped.agg({'tip' : np.max, 'size' : 'sum'}))

display(grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
                     'size': 'sum'}))

# Returning Aggregated Data Without Row Indexs: by passing as_index = false to groupby 

display(tips.groupby(['day','smoker'], as_index=False).mean())

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

C:\Users\JBarr\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

...

C:\Users\JBarr\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


tip_pct               total_bill            
            Durchschnitt Abeweichung Durchschnitt Abeweichung
day  smoker                                                  
Fri  No         0.151650    0.000791    18.420000   25.596333
     Yes        0.174783    0.002631    16.813333   82.562438
Sat  No         0.158048    0.001581    19.661778   79.908965
     Yes        0.147906    0.003767    21.276667  101.387535
Sun  No         0.160113    0.001793    20.506667   66.099980
     Yes        0.187250    0.023757    24.120000  109.046044
Thur No         0.160298    0.001503    17.113111   59.625081
     Yes        0.163863    0.001551    19.190588   69.808518

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [50]:
"""
# Apply: General split-apply-combine
The most general-purpose groupby method is apply.
apply splits the object being manipulated into pieces, invokes the passed fucntion on each piece,
and then attempts to concatenate the pieces together
"""

def top(df, n=5, columns='tip_pct'):
    return df.sort_values(by=columns)[-n:]

display(top(tips, n=6))

# Now, if we group by smoker, say and call apply with this function. we get the following 
# The top is called on each row group from the df and then the results are glued togehter abs
# using pandas.concat labeling the pieces with the group names
# The result therefore has hieratchial index, whose inner level contains index values from org df.

display(tips.groupby('smoker').apply(top))

display(tips.groupby(['smoker', 'day']).apply(top, n=1, columns='total_bill'))


,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

total_bill    tip     sex smoker   day    time  size   tip_pct
smoker day                                                                     
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4  0.115982

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()

display(result)

display(result.unstack('smoker'))

In [59]:
""" 
Quantile and Bucket Analysis: Random Sampling and Permutaion
Suppose you wanted to draw a random sample (with or without replacement)
from a large dataset for monte carlo simulation pusposes or some other application.
"""

# Hearts, Clubs, Diamonds

suits = ['H', 'S', 'C', 'D']

card_val = (list(range(1, 11)) + [10] * 3) * 4

base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'G']

cards = []

for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)


deck = pd.Series(card_val, index=cards)

display(deck)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
GH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
GS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
GC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
GD     10
dtype: int64